## Data Cleaning and Preparation

###### Handling Missing Data

The way that missing data is represented in pandas objects is somewhat imperfect,
but it is functional for a lot of users. For numeric data, pandas uses the floating-point
value NaN (Not a Number) to represent missing data. We call this a sentinel value that
can be easily detected

In [1]:
import pandas as pd
import numpy as np

In [3]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [7]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In [13]:
# The built-in Python None value is also treated as NA in object arrays
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

In [14]:
string_data.notnull()

0    False
1     True
2    False
3     True
dtype: bool

##### Filtering Out Missing Data

There are a few ways to filter out missing data. While you always have the option to
do it by hand using pandas.isnull and boolean indexing, the dropna can be helpful.
On a Series, it returns the Series with only the non-null data and index values

In [19]:
from numpy import nan as NA

In [20]:
data = pd.Series([1, NA, 3.5, NA, 7])

In [23]:
cleaned = data.dropna()
cleaned

0    1.0
2    3.5
4    7.0
dtype: float64

In [27]:
# This is equivalent to previous example. Here Boolean indexing is used.
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

With DataFrame objects, things are a bit more complex. You may want to drop rows
or columns that are all NA or only those containing any NAs. dropna by default drops
any row containing a missing value


In [33]:
data = pd.DataFrame([[1.,6.5, 3.], [1., NA, NA], [NA, NA, NA], [NA, 6.5, 3.]])
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [40]:
cleaned = data.dropna()
cleaned

,0,1,2
0,1.0,6.5,3.0


In [36]:
# Passing how='all' will only drop rows that are all NA
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [43]:
# To drop columns in the same way, pass axis=1:
data[4] = NA

In [44]:
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [45]:
data.dropna(how='all', axis=1)

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


A related way to filter out DataFrame rows tends to concern time series data. Suppose
you want to keep only rows containing a certain number of observations. We can
indicate this with the thresh argument

In [47]:
df = pd.DataFrame(np.random.randn(7,3))
df

,0,1,2
0,1.202119,0.437342,-0.083514
1,0.192650,-0.417913,1.931056
2,-0.360667,-1.721715,1.997239
3,-0.173243,-0.859095,-0.714570
4,1.048893,-0.423915,-0.364243
5,-0.095055,-0.602152,-1.053918
6,-0.349954,-0.252045,-0.422060


In [52]:
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df

,0,1,2
0,1.202119,NaN,NaN
1,0.192650,NaN,NaN
2,-0.360667,NaN,1.997239
3,-0.173243,NaN,-0.714570
4,1.048893,-0.423915,-0.364243
5,-0.095055,-0.602152,-1.053918
6,-0.349954,-0.252045,-0.422060


In [53]:
df.dropna()

,0,1,2
4,1.048893,-0.423915,-0.364243
5,-0.095055,-0.602152,-1.053918
6,-0.349954,-0.252045,-0.422060


In [61]:
df.dropna(thresh=2)

,0,1,2
2,-0.360667,NaN,1.997239
3,-0.173243,NaN,-0.714570
4,1.048893,-0.423915,-0.364243
5,-0.095055,-0.602152,-1.053918
6,-0.349954,-0.252045,-0.422060


In [59]:
df.dropna(axis=1)

,0
0,1.202119
1,0.192650
2,-0.360667
3,-0.173243
4,1.048893
5,-0.095055
6,-0.349954


##### Filling In Missing Data

In [62]:
# For most pur‐poses, the fillna method is the workhorse function to use
df

,0,1,2
0,1.202119,NaN,NaN
1,0.192650,NaN,NaN
2,-0.360667,NaN,1.997239
3,-0.173243,NaN,-0.714570
4,1.048893,-0.423915,-0.364243
5,-0.095055,-0.602152,-1.053918
6,-0.349954,-0.252045,-0.422060


In [65]:
df.fillna(0)

,0,1,2
0,1.202119,0.000000,0.000000
1,0.192650,0.000000,0.000000
2,-0.360667,0.000000,1.997239
3,-0.173243,0.000000,-0.714570
4,1.048893,-0.423915,-0.364243
5,-0.095055,-0.602152,-1.053918
6,-0.349954,-0.252045,-0.422060


In [66]:
# Calling fillna with a dict, we can use a different fill value for each column
df.fillna({1:0.5, 2:0})

,0,1,2
0,1.202119,0.500000,0.000000
1,0.192650,0.500000,0.000000
2,-0.360667,0.500000,1.997239
3,-0.173243,0.500000,-0.714570
4,1.048893,-0.423915,-0.364243
5,-0.095055,-0.602152,-1.053918
6,-0.349954,-0.252045,-0.422060


In [69]:
# fillna returns a new object, but we can modify the existing object in-place
df.fillna(0, inplace=True)
df

,0,1,2
0,1.202119,0.000000,0.000000
1,0.192650,0.000000,0.000000
2,-0.360667,0.000000,1.997239
3,-0.173243,0.000000,-0.714570
4,1.048893,-0.423915,-0.364243
5,-0.095055,-0.602152,-1.053918
6,-0.349954,-0.252045,-0.422060


In [82]:
# The same interpolation methods available for reindexing can be used with fillna
df = pd.DataFrame(np.random.rand(6,3))
df

,0,1,2
0,0.926834,0.297345,0.454495
1,0.548999,0.332209,0.242970
2,0.674743,0.585748,0.300526
3,0.672332,0.804326,0.063339
4,0.938367,0.639185,0.469232
5,0.106785,0.156610,0.313239


In [83]:
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df

,0,1,2
0,0.926834,0.297345,0.454495
1,0.548999,0.332209,0.242970
2,0.674743,NaN,0.300526
3,0.672332,NaN,0.063339
4,0.938367,NaN,NaN
5,0.106785,NaN,NaN


In [84]:
df.fillna(method='ffill')

,0,1,2
0,0.926834,0.297345,0.454495
1,0.548999,0.332209,0.242970
2,0.674743,0.332209,0.300526
3,0.672332,0.332209,0.063339
4,0.938367,0.332209,0.063339
5,0.106785,0.332209,0.063339


In [88]:
df.fillna(method='ffill', limit=2)

,0,1,2
0,0.926834,0.297345,0.454495
1,0.548999,0.332209,0.242970
2,0.674743,0.332209,0.300526
3,0.672332,0.332209,0.063339
4,0.938367,NaN,0.063339
5,0.106785,NaN,0.063339


In [91]:
# With fillna we can do lots of other things with a little creativity. We might pass the mean or median value of a Series
data = pd.Series([1., NA, 3.5, NA, 7])
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [92]:
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

### Data Transformation

###### Removing Duplicates

In [118]:
data = pd.DataFrame({'k1':['one', 'two'] * 3 + ['two'], 'k2': [1,1,2,3,3,4,4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [119]:
# The DataFrame method duplicated returns a boolean Series indicating whether each row is a duplicate or not
# Always look at the previous row
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [120]:
# Relatedly, drop_duplicates returns a DataFrame where the duplicated array is False
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


Both of these methods by default consider all of the columns; alternatively, we can
specify any subset of them to detect duplicates. Suppose we had an additional column
of values and wanted to filter duplicates only based on the 'k1' column

In [101]:
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [103]:
data.drop_duplicates(['k2'])

,k1,k2,v1
0,one,1,0
2,one,2,2
3,two,3,3
5,two,4,5


##### Transforming Data Using a Function or Mapping

In [107]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon', 'Pastrami', 
                              'corned beef', 'Bacon', 'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})

In [108]:
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [110]:
# Suppose we wanted to add a column indicating the type of animal that each food came from
meat_to_animal = {
    'bacon': 'pig',
    'pulled pork': 'pig',
    'pastrami': 'cow',
    'corned beef': 'cow',
    'honey ham': 'pig',
    'nova lox': 'salmon'
}

The map method on a Series accepts a function or dict-like object containing a map‐
ping, but here we have a small problem in that some of the meats are capitalized and
others are not. Thus, we need to convert each value to lowercase using the str.lower
Series method:

In [112]:
lowercased = data['food'].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [114]:
data['aniaml'] = lowercased.map(meat_to_animal)
data

,food,ounces,aniaml
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [116]:
# We could also have passed a function that does all the work
data['aniaml'] = data['food'].map(lambda x: meat_to_animal[x.lower()])

In [117]:
data

,food,ounces,aniaml
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


#### Replacing Values

In [121]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [123]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [124]:
# If you want to replace multiple values at once, you instead pass a list and then the substitute value
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [125]:
# To use a different replacement for each value, pass a list of substitutes
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [126]:
# The argument passed can also be a dict
data.replace({-999:np.nan, -1000:0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [141]:
data = pd.DataFrame(np.arange(12).reshape(3,4), index=['Ohio', 'Colorado', 'New York'], columns = ['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [142]:
# Like a Series, the axis indexes have a map method
transform = lambda x: x[:4].upper()
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [145]:
# We can assign to index, modifying the DataFrame in-place
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [152]:
# If we want to create a transformed version of a dataset without modifying the origi‐nal, a useful method is rename
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [159]:
# Notably, rename can be used in conjunction with a dict-like object providing new val‐ues for a subset of the axis label
data.rename(index={'OHIO': 'INDIANA'}, columns={'three' : 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [165]:
data.rename(index={'OHIO':'INDIANA'}, inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


##### Detecting and Filtering Outliers

In [168]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.head()

,0,1,2,3
0,0.207743,-1.125485,-1.914824,1.863503
1,-0.233698,-1.681371,-0.241588,-1.226003
2,0.135496,-0.857913,0.211172,-0.464452
3,-1.120969,-0.167035,-0.149717,-0.246277
4,-0.760788,0.344169,0.240787,-0.099535


In [169]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.053171,-0.004746,0.085923,0.007900
std,0.981277,1.040340,1.010942,0.966958
min,-3.281534,-3.222772,-3.413282,-3.850853
25%,-0.595531,-0.744714,-0.620973,-0.615410
50%,0.034851,-0.045126,0.093437,-0.013832
75%,0.720439,0.695062,0.834587,0.648520
max,3.134960,3.404370,3.306614,3.285587


In [186]:
col=data[2]
col[np.abs(col) > 3]

122   -3.413282
148    3.306614
Name: 2, dtype: float64

In [187]:
# To select all rows having a value exceeding 3 or –3, you can use the any method on a boolean DataFrame
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
13,-0.691888,3.045008,-0.559886,-1.098465
122,0.740751,-0.000061,-3.413282,1.806358
148,0.781807,-0.431107,3.306614,2.033305
157,0.657350,-1.158454,1.824342,-3.850853
246,-0.099032,-3.146763,-1.154720,-0.624399
276,-0.840930,3.308830,1.458556,0.757301
330,0.236077,0.715224,0.674091,3.095079
482,3.134960,0.782138,0.508254,-1.075095
527,0.309266,1.339308,0.279110,-3.464513
586,0.844343,-3.169447,1.020199,0.444453


In [191]:
# Values can be set based on these criteria. Here is code to cap values outside the inter‐val –3 to 3
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.053318,-0.004965,0.086030,0.008890
std,0.979944,1.036414,1.008660,0.960955
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.595531,-0.744714,-0.620973,-0.615410
50%,0.034851,-0.045126,0.093437,-0.013832
75%,0.720439,0.695062,0.834587,0.648520
max,3.000000,3.000000,3.000000,3.000000


The statement np.sign(data) produces 1 and –1 values based on whether the values
in data are positive or negative

In [193]:
np.sign(data).head()

,0,1,2,3
0,1.0,-1.0,-1.0,1.0
1,-1.0,-1.0,-1.0,-1.0
2,1.0,-1.0,1.0,-1.0
3,-1.0,-1.0,-1.0,-1.0
4,-1.0,1.0,1.0,-1.0


##### Computing Indicator/Dummy Variables

Another type of transformation for statistical modeling or machine learning applica‐
tions is converting a categorical variable into a “dummy” or “indicator” matrix. pandas has a get_dummies function
for doing this, though devising one yourself is not difficult. Let’s return to an earlier
example DataFrame

In [195]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'], 'data1': range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [197]:
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In some cases, we may want to add a prefix to the columns in the indicator Data‐
Frame, which can then be merged with the other data. get_dummies has a prefix argu‐
ment for doing this

In [207]:
dummies = pd.get_dummies(df['key'], prefix='key')

df_with_dummies = df[['data1']].join(dummies)
df_with_dummies

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


#### Regular Expressions

Regular expressions provide a flexible way to search or match (often more complex)
string patterns in text. The re module functions fall into three categories: pattern matching, substitution,
and splitting.

In [ ]:
# suppose we wanted to split a string with a variable number of whitespace characters
#(tabs, spaces, and newlines). The regex describing one or more whitespace characters is \s+

In [218]:
import re

In [222]:
text = "foo bar\t baz \tqux"
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

When we call re.split('\s+', text), the regular expression is first compiled, and
then its split method is called on the passed text. We can compile the regex yourself
with re.compile, forming a reusable regex object

In [223]:
regex = re.compile('\s+')
regex.split(text)

['foo', 'bar', 'baz', 'qux']

In [224]:
# If, instead, wewe wanted to get a list of all patterns matching the regex, you can use the findall method
regex.findall(text)

[' ', '\t ', ' \t']

Creating a regex object with re.compile is highly recommended if you intend to
apply the same expression to many strings; doing so will save CPU cycles

In [225]:
text = """
Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""

pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'


In [226]:
# # re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags = re.IGNORECASE)

In [228]:
# Using findall on the text produces a list of the email addresses
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

search returns a special match object for the first email address in the text. For the
preceding regex, the match object can only tell us the start and end position of the
pattern in the string

In [230]:
m = regex.search(text)
m

<re.Match object; span=(6, 21), match='dave@google.com'>

In [231]:
text[m.start():m.end()]

'dave@google.com'

## Data Wrangling: Join, Combine, and Reshape


In many applications, data may be spread across a number of files or databases or be
arranged in a form that is not easy to analyze. Here we focuses on tools to help
combine, join, and rearrange data.

##### Hierarchical Indexing

Hierarchical indexing is an important feature of pandas that enables us to have mul‐
tiple (two or more) index levels on an axis. Somewhat abstractly, it provides a way for
us to work with higher dimensional data in a lower dimensional form. Let’s start
with a simple example; create a Series with a list of lists (or arrays) as the index

In [245]:
data = pd.Series(np.random.rand(9), index = [['a','a','a','b','b','c','c','d','d'],
                                            [1,2,3,1,3,1,2,2,3]])
data

a  1    0.820030
   2    0.549760
   3    0.622240
b  1    0.782156
   3    0.726797
c  1    0.607044
   2    0.864420
d  2    0.475552
   3    0.206104
dtype: float64

In [234]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [236]:
data['b']

1    0.762405
3    0.993763
dtype: float64

In [238]:
data['b':'c']

b  1    0.762405
   3    0.993763
c  1    0.378184
   2    0.778541
dtype: float64

In [240]:
data.loc[['b', 'd']]

b  1    0.762405
   3    0.993763
d  2    0.373054
   3    0.974468
dtype: float64

In [243]:
# Selection is even possible from an “inner” level
data.loc[:, 2]

a    0.802792
c    0.778541
d    0.373054
dtype: float64

In [242]:
data

a  1    0.603317
   2    0.802792
   3    0.602947
b  1    0.762405
   3    0.993763
c  1    0.378184
   2    0.778541
d  2    0.373054
   3    0.974468
dtype: float64

In [ ]:
# Hierarchical indexing plays an important role in reshaping data and group-based
# operations like forming a pivot table.(Baki ase study korte hobe + stack and unstack niyeu study korte hobe)

### Hierarchical indexing need more study

### Combining and Merging Datasets

pandas.merge connects rows in DataFrames based on one or more keys. This
will be familiar to users of SQL or other relational databases, as it implements
database join operations.

###### Database-Style DataFrame Joins

Merge or join operations combine datasets by linking rows using one or more keys.
These operations are central to relational databases (e.g., SQL-based).

In [248]:
df1 = pd.DataFrame({'key': ['b','b','a','c','a','a','b'], 'data1':range(7)})

df2 = pd.DataFrame({'key' : ['a','b','d'], 'data2':range(3)})

In [249]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [250]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


This is an example of a many-to-one join; the data in df1 has multiple rows labeled a
and b, whereas df2 has only one row for each value in the key column. Calling merge
with these objects we obtain

In [251]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


Note that I didn’t specify which column to join on. If that information is not speci‐
fied, merge uses the overlapping column names as the keys. It’s a good practice to
specify explicitly

In [252]:
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


If the column names are different in each object, we can specify them separately

In [253]:
df3 = pd.DataFrame({'lkey' : ['b','b','a','c','a','a','b'], 'data1':range(7)})

df4 = pd.DataFrame({'rkey' : ['a','b','d'], 'data2':range(3)})

In [254]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


Here notice that the 'c' and 'd' values and associated data are missing from the
result. By default merge does an 'inner' join; the keys in the result are the intersec‐
tion, or the common set found in both tables.

Other possible options are 'left',
'right', and 'outer'. The outer join takes the union of the keys, combining the
effect of applying both left and right joins

In [258]:
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


In [259]:
pd.merge(df1, df2, how='left')

,key,data1,data2
0,b,0,1.0
1,b,1,1.0
2,a,2,0.0
3,c,3,NaN
4,a,4,0.0
5,a,5,0.0
6,b,6,1.0


In [260]:
pd.merge(df1, df2, how='right')

,key,data1,data2
0,a,2.0,0
1,a,4.0,0
2,a,5.0,0
3,b,0.0,1
4,b,1.0,1
5,b,6.0,1
6,d,NaN,2


Many-to-many merges have well-defined, though not necessarily intuitive, behavior.
Here’s an example

In [262]:
df1 = pd.DataFrame({'key': ['b','b','a','c','a','b'], 'data1':range(6)})

df2 = pd.DataFrame({'key' : ['a','b','a','b','d'], 'data2':range(5)})

In [263]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [264]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [265]:
pd.merge(df1, df2, on='key', how='left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


Many-to-many joins form the Cartesian product of the rows. Since there were three
'b' rows in the left DataFrame and two in the right one, there are six 'b' rows in the
result.

In [268]:
pd.merge(df1, df2, how='inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


In [270]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
 ....: 'key2': ['one', 'two', 'one'],
 ....: 'lval': [1, 2, 3]})
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


#  Over all this section (Data Wrangling: Join, Combine, and Reshape. . .)